In [1]:
from __future__ import print_function, division 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import sampler
import torchvision.datasets as dset
import torchvision.transforms as T
torch.set_default_tensor_type(torch.cuda.FloatTensor)
import numpy as np

import pandas as pd 
from torch.utils.data import Dataset, DataLoader 
from torchvision import transforms, utils
import os
import datetime 
import numpy as np 
import math

##文件数据行为: x   y   z  time     ；表示一个坐标点的三个坐标分量和 采集时间 ，使用空格符分隔

def pre(source, distance, diantance1):
    f = open(source, "r")  # 源文件
    fwrit = open(distance, "a")  # 卡2068
    for s in f.readlines():
        if len(s) == 1:
            fwrit.write(s)
        else:
            s = s[:-1]
            tem = s.split()
            re = tem[1].split(':')[1] + "\t" + tem[2].split(':')[1] + "\t" + tem[3].split(':')[1] + "\t" + tem[-2] + "\t" + tem[-1] + "\n"
            # if tem[0] != "2068":
            #     fwrit1.write(re)
            # else:
            if tem[0] == "2068":
                fwrit.write(re)
    f.close()
    fwrit.close()


def file_name(file_dir, target, target1):
    path = [file_dir + '\\' + x for x in os.listdir(file_dir)]
    for p in path:
        if not os.path.isdir(p):
            pre(p, target, target1)

def split_data(splot):
    """
    按照plot划分时间段
    """
    state = "D:\\fig\\data\\pre2068Static.txt"
    unrealize = "D:\\fig\\data\\pre2068Unrealize.txt"
    Sactive = "D:\\fig\\data\\pre2068Little.txt"
    Mactive = "D:\\fig\\data\\pre2068LargeMove.txt"
    files = [state, unrealize, Sactive, Mactive]
    mask = [0., 1., 2., 3.]
    splotre = []
    lable = [] 
    for index, file in enumerate(files):
        f = open(file, "r")
        mk = mask[index]

        flag = False 
        start = ''
        obj = [] 
        for s in f.readlines():
            if len(s) <= 1:
                continue
            else:
                
                s = s[:-1]
                seq = s.split("\t")
                if flag == False :
                    #本数据序列第一点的采集时间
                    start = 0
                    now = 0
                    flag = True 
                # 当前点的采集时间
                now = now + 1
                subt = now - start 
                obj.append(np.asarray(seq[:3],dtype='float64'))
            if subt > splot:
                splotre.append(np.asarray(obj))
                lable.append(mk)
                obj.clear() 
                flag = False
    splotre = np.asarray(splotre)
    lable = np.asarray(lable)
    return splotre, lable


In [2]:
# 大幅度运动 原始数据源
p = "E:\\datacollect\\trian\\active"
# 数据预处理结果保存路径
t = "D:\\fig\\data\\pre2068LargeMove.txt"  # 卡2068对应的数据，处理结果
t1 = "D:\\fig\\data\\preLargeMove.txt"  # 其他卡的处理结果

file_name(p, t, t1)
p = "E:\\datacollect\\trian\\little"  # 原始数据源

# 数据预处理结果保存路径
t = "D:\\fig\\data\\pre2068Little.txt"  # 卡2068对应的数据，处理结果
t1 = "D:\\fig\\data\\preLittle.txt"  # 其他卡的处理结果

file_name(p, t, t1)
p = "E:\\datacollect\\trian\\static"  # 原始数据源

# 数据预处理结果保存路径
t = "D:\\fig\\data\\pre2068Static.txt"  # 卡2068对应的数据，处理结果
t1 = "D:\\fig\\data\\preStatic.txt"  # 其他卡的处理结果
file_name(p, t, t1)
# 无意识运动，如转身，手摆动
p = "E:\\datacollect\\trian\\unrealize"  # 原始数据源
# 数据预处理结果保存路径
t = "D:\\fig\\data\\pre2068Unrealize.txt"  # 卡2068对应的数据，处理结果
t1 = "D:\\fig\\data\\preUnrealize.txt"  # 其他卡的处理结果径
file_name(p, t, t1)

In [3]:
splotre, lable = split_data(29)

In [4]:
splotre.shape

(19088, 30, 3)

In [5]:
lable

array([0., 0., 0., ..., 3., 3., 3.])

In [41]:
NUM_TRAIN = 12000
np.random.shuffle(index)
splotre = splotre[index]
lable = lable[index]
train_splot = splotre[:NUM_TRAIN-3000]
train_lable = lable[:NUM_TRAIN-3000]
val_splot = splotre[NUM_TRAIN-3000:NUM_TRAIN]
val_lable = lable[NUM_TRAIN-3000:NUM_TRAIN]
test_splot = splotre[NUM_TRAIN:]
test_lable = lable[NUM_TRAIN:]


In [42]:
class ToTensor(object):
    def __call__(self, splot):
        return torch.from_numpy(splot)
    
trans = T.Compose([
    ToTensor()
])

class LocationDataset(Dataset):
    def __init__(self, splotre, lable, transform = trans):
        self.splotre = np.transpose(splotre, (0, 2, 1))
        self.lable = lable 
        self.transform = transform 
    
    def __len__(self):
        return len(self.splotre)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        splot = self.splotre[idx] 
        lable = self.lable[idx]
        tensor = trans(splot)
        return tensor, lable
train_dataset = LocationDataset(train_splot, train_lable)
loader_train = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataset = LocationDataset(val_splot, val_lable)
loader_val = DataLoader(val_dataset, batch_size=4, shuffle=True)


In [43]:
loader_val.dataset[2]

(tensor([[ 7.3320e+00,  7.3341e+00,  7.2880e+00,  7.1911e+00,  7.1709e+00,
           7.2502e+00,  7.3349e+00,  7.4073e+00,  7.5008e+00,  7.5851e+00,
           7.6368e+00,  7.7193e+00,  7.8462e+00,  7.8823e+00,  7.9330e+00,
           8.0516e+00,  8.0717e+00,  8.0458e+00,  8.0339e+00,  7.9365e+00,
           7.9232e+00,  7.9887e+00,  7.9887e+00,  7.9756e+00,  7.9348e+00,
           7.9348e+00,  7.8910e+00,  7.7736e+00,  7.6801e+00,  7.6358e+00],
         [ 3.1841e-01,  1.2529e-01,  7.4530e-02, -6.9990e-02, -6.0800e-02,
          -4.4900e-03,  1.0128e-01,  2.6819e-01,  4.5018e-01,  6.4281e-01,
           8.4811e-01,  1.0025e+00,  1.1297e+00,  1.3010e+00,  1.4925e+00,
           1.6707e+00,  1.8937e+00,  2.0383e+00,  2.2277e+00,  2.4052e+00,
           2.5926e+00,  2.7865e+00,  2.7865e+00,  3.1161e+00,  3.3097e+00,
           3.3097e+00,  3.3666e+00,  3.1911e+00,  2.9938e+00,  2.7971e+00],
         [ 1.5000e+00,  1.5000e+00,  1.5000e+00,  1.5000e+00,  1.5000e+00,
           1.5000e+00, 

In [44]:
def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

In [45]:
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [46]:
USE_GPU = False

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cpu


In [47]:
import torch.nn.functional as F  # useful stateless functions
def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()

In [48]:
def check_accuracy_part34(loader, model): 
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [52]:
channel_1 = 32 
channel_2 = 16
learning_rate = 1e-4
model = None 
optimizer = None 

model = nn.Sequential(
    nn.Conv1d(3, 32, 3, stride = 1, padding = 1),
    nn.ReLU(),
    nn.Conv1d(32, 64, 2, stride = 2, padding = 5),
    nn.ReLU(),
    nn.Conv1d(64, 128, 5, stride = 1),
    nn.ReLU(),
    Flatten(),
    nn.Linear(2048, 4)
)
optimizer = optim.SGD(model.parameters(), lr = learning_rate, momentum=0.9, nesterov = True)
train_part34(model, optimizer, 3)

Iteration 0, loss = 1.4089
Got 871 / 3000 correct (29.03)

Iteration 100, loss = 1.4315
Got 652 / 3000 correct (21.73)

Iteration 200, loss = 1.3567
Got 885 / 3000 correct (29.50)

Iteration 300, loss = 1.3585
Got 1349 / 3000 correct (44.97)

Iteration 400, loss = 1.3241
Got 1571 / 3000 correct (52.37)

Iteration 500, loss = 1.2818
Got 1580 / 3000 correct (52.67)

Iteration 600, loss = 1.2061
Got 1227 / 3000 correct (40.90)

Iteration 700, loss = 1.2959
Got 1777 / 3000 correct (59.23)

Iteration 800, loss = 1.1423
Got 1902 / 3000 correct (63.40)

Iteration 900, loss = 1.2256
Got 1925 / 3000 correct (64.17)

Iteration 1000, loss = 0.9346
Got 2174 / 3000 correct (72.47)

Iteration 1100, loss = 1.4562
Got 1729 / 3000 correct (57.63)

Iteration 1200, loss = 1.4042
Got 2135 / 3000 correct (71.17)

Iteration 1300, loss = 1.2985
Got 2151 / 3000 correct (71.70)

Iteration 1400, loss = 0.6270
Got 2231 / 3000 correct (74.37)

Iteration 1500, loss = 0.9422
Got 2249 / 3000 correct (74.97)

Iterati

In [53]:
test_dataset = LocationDataset(test_splot, test_lable)
loader_test = DataLoader(test_dataset)

In [54]:
def test(loader,model):
    num_correct = 0 
    num_samples = 0 
    res = []
    for x, y in loader:
        x = x.to(device=device, dtype=dtype)
        y = y.to(device=device, dtype=torch.long)
        scores = model(x)
        _, preds = scores.max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
        res.append(preds.item())
    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return res


res = test(loader_test,model)

from sklearn import metrics

def gradeOfClassifier():
    P = metrics.precision_score(test_lable, res, average='macro')
    R = metrics.recall_score(test_lable, res, average='macro')
    # F1分数
    F1 = metrics.f1_score(test_lable, res, average='weighted')
    # 混淆矩阵
    M = metrics.confusion_matrix(test_lable, res, labels=[0.0, 1.0, 2.0, 3.0])
    print("查准率" + str(P))
    print("召回率" + str(R))
    print("F1分数" + str(F1))
    print("混淆矩阵")
    print(M)
    print()
    return P
gradeOfClassifier()

Got 6337 / 7088 correct (89.40)
查准率0.8955945742545255
召回率0.8950398425453393
F1分数0.8959379616377634
混淆矩阵
[[1870    4  148   48]
 [   6 1511   26   30]
 [  22    9 1335  189]
 [  16   11  242 1621]]



0.8955945742545255

In [55]:
import matplotlib.pyplot as plt 
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号


In [56]:
from lys.filter.examples.sine_wave import kalmanfilter
from lys.filter.movinAverage import MoveAverage
from lys.filter.sampleFilter import last_polyfit
from lys.evaluate import getMae, Mape, Rmse

def classifer(model, splot, path):
    resultSeqStatic = []  # 静止状态
    resultSeqUnrealized = []  # 无意识运动状态
    resultSeqBackMove = []  # 短距离往返运动
    resultSeqforward = []  # 长距离往返运动
    splotre = []
    # 运动状态分析，与剔除
    with open(path, 'r') as f:
        obj = []
        flag = False  # 是否第一次记录起始时间
        start = ''  # 当前子序列的起始时间
        for s in f.readlines():  # 按行读取数据
            if len(s) <= 1:
                continue
            else:
                
                s = s[:-1]
                seq = s.split(" ")
                if flag == False :
                    #本数据序列第一点的采集时间
                    start = 0
                    now = 0
                    flag = True 
                # 当前点的采集时间
                now = now + 1
                subt = now - start 
                myseq = [seq[1][7:], seq[2][7:], seq[3][7:]]
                obj.append(np.asarray(myseq,dtype='float64'))
            if subt > splot:
                splotre.append(np.asarray(obj))
                obj.clear() 
                flag = False
    splotre = np.asarray(splotre)
    for plot in splotre:
        tensor = torch.FloatTensor(plot.T)
        tensor = tensor.view((1,3,30))
        scores = model(tensor)
        _, state = scores.max(1)
        if state.item() == 0:
            resultSeqStatic += plot.tolist()
        elif state.item() == 1:
            resultSeqUnrealized += plot.tolist()
        elif state.item() == 2:
            resultSeqBackMove += plot.tolist()
        elif state.item() == 3:
            resultSeqforward += plot.tolist()
    return resultSeqStatic, resultSeqUnrealized, resultSeqBackMove, resultSeqforward

In [57]:
def countdistance(seq):
    distance = 0.0
    # 计算一个序列的运动距离
    if len(seq) <= 1:  # 只有一个点时，距离为零
        return distance
    else:
        p1 = seq[0]  # 轨迹的第一个点
        for value in seq[1:]:
            p2 = value
            d = np.sqrt((float(p1[0]) - float(p2[0])) ** 2 + (float(p1[1]) - float(p2[1])) ** 2)  # 两点间的距离
            if d < 2.0:
                distance += d
            p1 = p2
        distance = float("%.2f" % distance)
        return distance


# 卡尔曼滤波器
def kalmanFilter(seq):
    x_value = []
    y_value = []
    for i in range(len(seq)):
        temp = seq[i]
        x_value.append(float(temp[0]))
        y_value.append(float(temp[1]))
    x_value = kalmanfilter(x_value, len(x_value))  # 滤波

    length = min(len(x_value), len(y_value))  # 数组长度
    sequence = [[x_value[i], y_value[i]] for i in range(0, length)]
    return sequence


# 最小二乘法拟合多项式
def leastfit(seq):
    result = last_polyfit(seq)
    return result


def MovingAverage(seq):
    obj = MoveAverage(seq, 11)  # 创建对象
    result = obj.smooth()  # 平滑数据

    return result


In [58]:
dirPath = "D:\\fig\\test\\data"
path = [dirPath + '\\' + x for x in os.listdir(dirPath)]
resultSeqStatic, resultSeqUnrealized, resultSeqBackMove, resultSeqforward = classifer(model, 29, path[0])
print(countdistance(resultSeqStatic))
print(countdistance(resultSeqUnrealized))
print(countdistance(resultSeqBackMove))
print(countdistance(resultSeqforward))
countdistance(resultSeqBackMove) + countdistance(resultSeqUnrealized) + countdistance(resultSeqforward)

0.5
0.0
3.23
0.0


3.23

In [ ]:
# index=1  卡尔曼滤波
# index=2  最小二乘
# index=1  滑动均值
def deal(index, seq):
    if index == 1:
        return kalmanFilter(seq)
    elif index == 2:
        return leastfit(seq)
    elif index == 3:
        return MovingAverage(seq)

In [ ]:
# 对不同状态的数据处理
def fn(i, j, k, classfilter, slot, path):
    re = []
    for p in path:
        # 各状态下的轨迹
        resultSeqStatic, resultSeqUnrealized, resultSeqBackMove, resultSeqforward = classifer(classfilter,
                                                                                              slot, p)
        # #除去静止状态后，在拟合处理数据的轨迹的距离
        d1 = countdistance(deal(i, resultSeqUnrealized))
        d2 = countdistance(deal(j, resultSeqBackMove))
        d3 = countdistance(deal(k, resultSeqforward))
        distance2 = d1 + d2 + d3
        re.append(distance2)
    return re


def draw2(title, measure, trueValue):
    plt.figure(title)
    x = [i for i in range(1, 13)]
    plt.xlabel('轨迹序号', fontproperties='SimHei', fontsize=10)
    plt.ylabel('轨迹长度(米)', fontproperties='SimHei', fontsize=10)
    plt.title(title)

    plt.plot(x, measure, marker='+', linestyle='--', label='1d-CNN')  # 计算值

    plt.plot(x, trueValue, marker='o', linestyle='-', label='真实距离')  # 真值
    plt.legend(loc='lower right')  # label生效
    plt.xticks(x)
    plt.show()

In [ ]:
def main2(slot):
    dirPath = "D:\\项目\\廖煜胜-论文\\论文\\数据与程序\\datacollect\\test\\data"
    path = [dirPath + '\\' + x for x in os.listdir(dirPath)]
    i = 1
    j = 2
    k = 1
    re = fn(i, j, k, model, slot, path)

    return re

In [ ]:
def getvalue(path):
    f = open(path, "r")  # 源文件
    result = []
    for s in f.readlines():
        num = float(s.split("  ")[1][:-1])
        result.append(num)
    return result


# 求和
def distanceofseq(value):
    val_sum = 0
    for i in range(len(value)):
        val_sum += float(value[i])
    return val_sum


def SD(measure, value):
    d = 0  # 标准差
    err = 0  # 平均误差
    sum_value = 0
    sum_measure = 0
    for i in range(len(value)):
        d = d + (value[i] - measure[i]) ** 2
        err += measure[i] - value[i]
        sum_value += value[i]
        sum_measure += measure[i]

    d = np.sqrt(d) / len(value)
    err = err / len(value)
    ff = (sum_measure - sum_value) / sum_value

    print("误差： " + str(err))
    print("相对误差率： " + str(ff))
    print("标准差： " + str(d))
    print()

In [ ]:
path = "D:\\项目\\廖煜胜-论文\\论文\\数据与程序\\datacollect\\test\\truevalue.txt"
value = getvalue(path)  # 真值
measure = main2(29)
getMae(measure, value)
Mape(measure, value)
Rmse(measure, value)

In [ ]:
def draw4(title, measure, trueValue, label):
    plt.figure(title)
    x = [i for i in range(1, 13)]
    plt.xlabel('轨迹序号', fontproperties='SimHei', fontsize=10)
    plt.ylabel('轨迹长度（米）', fontproperties='SimHei', fontsize=10)
    plt.title("计算值与真实值对比")

    plt.subplots_adjust(bottom=0.15)  # 图边距t
    plt.subplots_adjust(left=0.15)
    plt.xticks(x)
    for i in range(1, len(x)):
        d = ((measure[i] - trueValue[i]) / trueValue[i]) * 100
        text = float("%.2f" % d)
        plt.text(x[i] - 0.4, measure[i] + 30, str(text) + "%", fontdict={'size': '10'})

    plt.plot(x, measure, marker='*', linestyle='-.', label=label)  # 计算值
    plt.plot(x, trueValue, marker='o', linestyle='-', label='真实值')  # 真值
    plt.legend(loc='lower right')  # label生效
    plt.savefig("D:\\fig\\picture\\" + str(title) + ".png")

In [ ]:
draw4("1d-cnn", measure, value, "1d-cnn")